# original data

In [2]:
import pandas as pd
import numpy as np

step to create the data:
- make sure that in the folder `raw_data` there is a file `NYCTAXI202401-202403_OD.zip`
- cd to the folder `raw_data`
- linux shell `unzip NYCTAXI202401-202403_OD.zip`
- run the following notebook
- the output will be saved in the folder `data`, which will have two folders `total` and `timeperiod`.

In [3]:
import os
# if not os.path.exists("data"):
    # os.makedirs("data")
# if not os.path.exists("data/timeperiod"):
    # os.makedirs("data/timeperiod")
# if not os.path.exists("data/total"):
    # os.makedirs("data/total")

In [4]:
geo = pd.read_csv('raw_data/NYCTAXI202401-202403_OD/NYCTAXI202401-202403.geo')
print(geo.shape)
geo.head( )

(263, 3)


,geo_id,type,coordinates
0,0,Polygon,"[[['-74.18445300', '40.69499600'], ['-74.18563..."
1,1,MultiPolygon,"[[[['-73.81768667', '40.61493300'], ['-73.8165..."
2,2,Polygon,"[[['-73.84240629', '40.86847474'], ['-73.84280..."
3,3,Polygon,"[[['-73.97464721', '40.71884235'], ['-73.97347..."
4,4,Polygon,"[[['-74.17225508', '40.56170484'], ['-74.17348..."


In [5]:
geo["type"].value_counts()

Polygon         240
MultiPolygon     23
Name: type, dtype: int64

In [6]:
rel = pd.read_csv('raw_data/NYCTAXI202401-202403_OD/NYCTAXI202401-202403.rel')
print(rel.shape)
rel.head()

(69169, 5)


,rel_id,type,origin_id,destination_id,cost
0,0,geo,0,0,0.0
1,1,geo,0,1,31349.0
2,2,geo,0,2,33747.0
3,3,geo,0,3,17205.0
4,4,geo,0,4,14927.0


In [7]:
od = pd.read_csv('raw_data/NYCTAXI202401-202403_OD/NYCTAXI202401-202403.od')#.drop(["type"],axis=1)
print(od.shape)
od.head()

(150995927, 6)


,dyna_id,type,time,origin_id,destination_id,flow
0,0,state,2024-01-01T00:00:00Z,0,0,0.0
1,1,state,2024-01-01T01:00:00Z,0,0,0.0
2,2,state,2024-01-01T02:00:00Z,0,0,0.0
3,3,state,2024-01-01T03:00:00Z,0,0,0.0
4,4,state,2024-01-01T04:00:00Z,0,0,0.0


In [8]:
# od["flow"].value_counts().to_frame().reset_index()

# Get two kind of network

## Network group by time interval

In [10]:
od["time"] = pd.to_datetime(od["time"])

od["time"] = od["time"].dt.tz_convert('America/New_York')

od["hour"] = od["time"].dt.hour
od

,dyna_id,type,time,origin_id,destination_id,flow,hour
0,0,state,2023-12-31 19:00:00-05:00,0,0,0.0,19
1,1,state,2023-12-31 20:00:00-05:00,0,0,0.0,20
2,2,state,2023-12-31 21:00:00-05:00,0,0,0.0,21
3,3,state,2023-12-31 22:00:00-05:00,0,0,0.0,22
4,4,state,2023-12-31 23:00:00-05:00,0,0,0.0,23
...,...,...,...,...,...,...,...
150995922,150995922,state,2024-03-31 14:00:00-04:00,262,262,0.0,14
150995923,150995923,state,2024-03-31 15:00:00-04:00,262,262,0.0,15
150995924,150995924,state,2024-03-31 16:00:00-04:00,262,262,0.0,16
150995925,150995925,state,2024-03-31 17:00:00-04:00,262,262,0.0,17


In [11]:
od["hour"].value_counts()

19    6294379
20    6294379
17    6294379
16    6294379
15    6294379
14    6294379
13    6294379
12    6294379
11    6294379
10    6294379
9     6294379
8     6294379
7     6294379
6     6294379
5     6294379
4     6294379
3     6294379
1     6294379
0     6294379
23    6294379
22    6294379
21    6294379
18    6294379
2     6225210
Name: hour, dtype: int64

69169的od对

In [12]:
# Define the boundaries of the time periods
bins = [-0.1, 0.1, 6, 10, 12, 16, 19, 23]

# Define the labels for the time periods
labels = ['Early Morning','Early Morning', 'Morning Rush', 'Morning to Noon', 'Afternoon', 'Evening Rush', 'Night']
# num_labels = [0,1,2,3,4,5,0]

tmp = od.copy(deep =True)
tmp['time_period'] = pd.cut(tmp['hour'], bins=bins, labels=labels,ordered=False)
# tmp["time_period"] = tmp["time_period"].astype(str)

In [13]:
# tmp["time_period"].isnull().sum()

In [14]:
tmp.drop(["hour","dyna_id","type"],axis=1,inplace=True)
tmp

,time,origin_id,destination_id,flow,time_period
0,2023-12-31 19:00:00-05:00,0,0,0.0,Evening Rush
1,2023-12-31 20:00:00-05:00,0,0,0.0,Night
2,2023-12-31 21:00:00-05:00,0,0,0.0,Night
3,2023-12-31 22:00:00-05:00,0,0,0.0,Night
4,2023-12-31 23:00:00-05:00,0,0,0.0,Night
...,...,...,...,...,...
150995922,2024-03-31 14:00:00-04:00,262,262,0.0,Afternoon
150995923,2024-03-31 15:00:00-04:00,262,262,0.0,Afternoon
150995924,2024-03-31 16:00:00-04:00,262,262,0.0,Afternoon
150995925,2024-03-31 17:00:00-04:00,262,262,0.0,Evening Rush


In [15]:
tmp_total = tmp.groupby(["origin_id","destination_id"]).agg({"flow":"mean"}).reset_index()
tmp_total[tmp_total["flow"]!=0].reset_index(drop = True).to_csv("total/NYCTAXI_OD_Total.csv",index=False)

In [16]:
tmp.groupby("time_period").size()

time_period
Afternoon          25177516
Early Morning      43991484
Evening Rush       18883137
Morning Rush       25177516
Morning to Noon    12588758
Night              25177516
dtype: int64

In [17]:
# tmp_nomissing = tmp.loc[tmp["flow"] != 0,:].reset_index(drop=True)
# tmp_nomissing

In [18]:
# tmp_group = tmp.loc[tmp["time_period"] == "Evening Rush",:]
# tmp_group = tmp_group.groupby(["origin_id","destination_id"]).agg({"flow":"mean"}).reset_index()
# tmp_group[tmp_group["flow"]!=0].reset_index(drop = True)

In [19]:
for i in labels:
    tmp_group = tmp.loc[tmp["time_period"] == i,:]
    tmp_group = tmp_group.groupby(["origin_id","destination_id"]).agg({"flow":"mean"}).reset_index()    
    tmp_group[tmp_group["flow"]!=0].reset_index(drop = True).to_csv(f"timeperiod/NYCTAXI_OD_{i}.csv")
    # display(tmp_group)

In [20]:
# od_flow_nozero = od[od["flow"]>0].reset_index(drop=True)

# # Define the boundaries of the time periods
# bins = [0, 7, 9, 12, 14, 17, 19, 24]

# # Define the labels for the time periods
# labels = ['Night', 'Morning Rush', 'Morning', 'Noon', 'Afternoon', 'Evening Rush', 'Night']
# # num_labels = [0,1,2,3,4,5,0]

# # Create a new column that groups hours into time periods
# od_flow_nozero['time_period'] = pd.cut(od_flow_nozero['hour'], bins=bins, labels=labels, right=False,ordered=False)

# # Use the groupby function to group by the new column
# od_flow_nozero_agg = od_flow_nozero.groupby(["time_period","origin_id","destination_id"]).agg({"flow": ["count","sum","mean"]}).reset_index()

# od_flow_nozero_agg = od_flow_nozero_agg.loc[od_flow_nozero_agg["flow"]["count"]!=0,:]

# od_flow_nozero_f = od_flow_nozero_agg.copy(deep=True)
# od_flow_nozero_f.columns = ["time_period","origin_id","destination_id","count","sum","mean"]
# od_flow_nozero_f.drop(["count","sum"],axis=1,inplace=True)
# od_flow_nozero_f.rename(columns={"mean":"mean_flow"},inplace = True)
# od_flow_nozero_f



## Total flow network

agg od by time   -> data/NYCTAXI_OD_agg.csv

In [21]:
od_nomissing = od.loc[od["flow"]!=0, :]
od_nomissing.reset_index(drop=True,inplace=True)
od_nomissing

,dyna_id,type,time,origin_id,destination_id,flow,hour
0,104371,state,2024-03-14 14:00:00-04:00,0,47,1.0,14
1,104386,state,2024-03-15 05:00:00-04:00,0,47,1.0,5
2,217791,state,2024-03-10 14:00:00-04:00,0,99,1.0,14
3,408096,state,2024-03-26 14:00:00-04:00,0,186,1.0,14
4,456032,state,2024-03-22 20:00:00-04:00,0,208,1.0,20
...,...,...,...,...,...,...,...
2595913,150993739,state,2024-03-31 14:00:00-04:00,262,261,3.0,14
2595914,150993740,state,2024-03-31 15:00:00-04:00,262,261,7.0,15
2595915,150993741,state,2024-03-31 16:00:00-04:00,262,261,3.0,16
2595916,150993742,state,2024-03-31 17:00:00-04:00,262,261,2.0,17


In [22]:
od_agg = od_nomissing.groupby(["origin_id","destination_id"]).agg({"flow":"mean"}).reset_index()
od_agg


,origin_id,destination_id,flow
0,0,47,1.000000
1,0,99,1.000000
2,0,186,1.000000
3,0,208,1.000000
4,0,230,1.000000
...,...,...,...
25966,262,257,1.250000
25967,262,258,1.000000
25968,262,259,1.054054
25969,262,260,1.093525


In [23]:
# od_agg.to_csv("total/NYCTAXI_OD_Total.csv",index=False)

join od_agg and rel  --- not used

In [24]:
# od_agg = pd.read_csv("data/NYCTAXI_OD_agg.csv")
# od_f = od_agg.copy(deep = True)

# od_f["cost"] = rel["cost"]

In [25]:
# od_f.to_csv("total/NYCTAXI_OD_Total.csv",index=False)

In [26]:
# od_f.head()

## add location to the od_f --- not used

In [27]:
# od_f_merge = pd.merge(od_f,geo,left_on="origin_id",right_on="geo_id",how="left").drop(["geo_id","type"],axis=1)\
#     .rename(columns={"coordinates":"origin_coordinates"})
# od_f_merge

In [28]:
# od_f_merge = pd.merge(od_f_merge,geo,left_on="destination_id",right_on="geo_id",how="left").drop(["geo_id","type"],axis=1)\
#     .rename(columns={"coordinates":"destination_coordinates"})
# od_f_merge